# Learn

In [1]:
import json
import os
from pathlib import Path
import sys
import time

import joblib
import numpy as np
import sklearn.ensemble
import sklearn.feature_selection
import sklearn.metrics
import sklearn.model_selection
from tqdm.auto import tqdm

root = str(Path.cwd().parents[1])
sys.path.append(root)
os.chdir(root)
import src

In [ ]:
X, y, s = src.get_Xys()
y = y==1
ys = y + 3*s

skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
folds = list(skf.split(X=X, y=ys))

iterator = [
    (orthogonality, int(max_depth), n_bins, fold, train_idx, test_idx) 
    for n_bins in [2]
    for orthogonality in [0]
    for max_depth in np.arange(1, 11)
    for fold, (train_idx, test_idx) in enumerate(folds)
]

count = 0
for ortho, max_depth, n_bins, fold, train_idx, test_idx in tqdm(iterator, 
                                                                mininterval=0, 
                                                                miniters=1):
    assert isinstance(max_depth, int)
    assert isinstance(n_bins, int)
    filepath = f'cache/FRF_v3_C/{max_depth}-{ortho:.1f}-{fold}-{n_bins}'
    if not os.path.isfile(filepath + '.pkl'):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        s_train, s_test = s[train_idx], s[test_idx]        
        
        vt = sklearn.feature_selection.VarianceThreshold()
        vt.fit(X_train)
        X_train = vt.transform(X_train)
        X_test = vt.transform(X_test)
        
        clf = src.FairRandomForestClassifier(
            max_depth=max_depth, 
            n_estimators=100, 
            orthogonality=ortho, 
            n_jobs=100, 
            n_bins=n_bins
        ) 
        start = time.time()
        clf.fit(X_train, y_train, s_train)
        joblib.dump(clf, filepath + '.pkl')
        fit_time = time.time() - start                   

# Predict

In [ ]:
import os

import joblib
import numpy as np
import sklearn.feature_selection
import sklearn.model_selection
from tqdm.auto import tqdm

import src

X, y, s = src.get_Xys()
y = y==1
ys = y+3*s

skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
folds = list(skf.split(X=X, y=ys))

iterator = [
    (orthogonality, int(max_depth), n_bins, fold, train_idx, test_idx) 
    for orthogonality in [0]
    for max_depth in np.arange(1, 11)
    for n_bins in [2]
    for fold, (train_idx, test_idx) in enumerate(folds)
]

for ortho, max_depth, n_bins, fold, train_idx, test_idx in tqdm(iterator):
    assert isinstance(max_depth, int)
    assert isinstance(n_bins, int)
    filepath = f'cache/FRF_v3_C/{max_depth}-{ortho:.1f}-{fold}-{n_bins}'

    if not os.path.isfile(filepath + '.npy'):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        s_train, s_test = s[train_idx], s[test_idx]

        vt = sklearn.feature_selection.VarianceThreshold()
        vt.fit(X_train)
        X_train = vt.transform(X_train)
        X_test = vt.transform(X_test) 

        assert os.path.isfile(filepath + '.pkl')
        clf = joblib.load(filepath + '.pkl')
        y_score = clf.predict_proba(X_test)[:,1] #type: ignore
        np.save(filepath + '.npy', y_score)

# Measure

In [ ]:
import json
import os

import joblib
import numpy as np
import sklearn.model_selection
from tqdm.auto import tqdm

import src

overwrite=True

X, y, s = src.get_Xys()
y = y==1
ys = y + 3*s

skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
folds = list(skf.split(X=X, y=ys))

iterator = [
    (ortho, int(max_depth), n_bins, fold, train_idx, test_idx) 
    for ortho in [0]
    for max_depth in np.arange(1, 11)
    for n_bins in [2]
    for fold, (train_idx, test_idx) in enumerate(folds)
]

results = list()

for ortho, max_depth, n_bins, fold, train_idx, test_idx in tqdm(iterator):
    assert isinstance(max_depth, int)
    assert isinstance(n_bins, int)
    filepath = f'cache/FRF_v3_C/{max_depth}-{ortho:.1f}-{fold}-{n_bins}'

    if not os.path.isfile(filepath + '.json') or overwrite:
        y_test, s_test = y[test_idx], s[test_idx]

        assert os.path.isfile(filepath + '.npy')
        y_score = np.load(filepath + '.npy')
        
        measures = src.get_measures(y_test, y_score, s_test)
        
        result_this_run_ = {
            'fold': fold,
            'max_depth': max_depth,
            'orthogonality': ortho,
            'n_bins': n_bins,
            **measures
        }
        with open(filepath + '.json', 'w') as file:
            json.dump(result_this_run_, file)
    else:
        with open(filepath + '.json') as file:
            result_this_run_ = json.load(file)
    results.append(result_this_run_)
joblib.dump(results, 'cache/FRF_v3_C_non-nested.pkl')

# Analysis

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.model_selection

import src

data = (
    pd.DataFrame(pd.read_pickle("cache/FRF_v3_C_non-nested.pkl"))
    .groupby(['max_depth', 'orthogonality', 'n_bins'])
    .mean()
    .reset_index()
    .sort_values('auc_y', ascending=False)
    .drop_duplicates(subset=['orthogonality'])
    .sort_values('orthogonality')
)
data

X, y, s = src.get_Xys()
y = y<2
title_prefix = f'Fair Random Forest (nested gridsearch) | Scenario C\nmax_depth=[1-10] | n_estimators=100 | n_bins=2'
base_rate = np.mean(y)
base_rate_black = np.mean(y[s])
base_rate_white = np.mean(y[~s])

In [3]:
data

,max_depth,orthogonality,n_bins,fold,auc_y,auc_s,threshold,proportion_predicted_ok_white,proportion_predicted_ok_black,disparate_impact,disparate_impact_epsilon,false_ok_white,false_ok_black,true_ok_white,true_ok_black,equalized_odds_false_ok,equalized_odds_true_ok,equalized_odds_epsilon
5,6,0,2,2,0.760377,0.677286,0.645302,0.253412,0.496591,1.960233,-0.960233,0.155477,0.428546,0.535371,0.5863,0.273069,0.057568,0.273069
